In [ ]:
!pip install unsloth transformers datasets accelerate trl peft bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.5/218.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31

In [ ]:
# Gerekli kütüphaneler (Unsloth en üstte olmalı!)
import unsloth
from unsloth import FastLanguageModel
import torch
import json
import gc
import time
from datetime import datetime, timedelta

# Model ve tokenizer'ı Unsloth üzerinden yüklüyoruz
model_path = "/content/drive/Shareddrives/Senior Design Project/fine_tuned_model_outputs/llama3.2-3b/llama3.2_3B_3epoch_5batch"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=16000,
    dtype=torch.float16,
    load_in_4bit=True,
    device_map="auto"
)

# JSON dosyasını yüklüyoruz
with open("/content/drive/Shareddrives/Senior Design Project/ShareGPT dataset Fine Tune kodu  test dataseti/second_200_papers_test.json", "r", encoding="utf-8") as file:
    articles = json.load(file)

# System prompt
system_instruction = (
    "You are an expert academic writer specializing in generating high-quality abstracts for scholarly publications.\n\n"
    "Your task is to produce a concise abstract (150-250 words) based on a research paper's title and content. Please ensure the abstract:\n"
    "- Clearly states the research problem or question,\n"
    "- Summarizes the methodology or approach,\n"
    "- Highlights the key findings,\n"
    "- Explains the significance or implications of the results.\n\n"
    "Use formal, objective language appropriate for academic journals. Avoid unnecessary jargon and maintain clarity for a broad scholarly audience."
)

# Abstract üretim fonksiyonu
def generate_abstract_full_text(article):
    title = article["title"]
    content = article["content"]

    user_prompt = (
        "Generate an academic abstract based on the following paper:\n\n"
        f"Title: {title}\n\n"
        f"Content: {content}\n\n"
        "Please ensure the abstract:\n"
        "- Is concise (150-250 words),\n"
        "- Clearly states the research problem,\n"
        "- Summarizes the methodology,\n"
        "- Highlights the key findings,\n"
        "- Explains the significance or implications,\n"
        "- Uses formal academic language suitable for a scholarly journal,\n"
        "- Avoids unnecessary jargon while maintaining clarity."
    )

    messages = [
        {"role": "system", "content": system_instruction},
        {"role": "user", "content": user_prompt}
    ]

    # Prompt formatlama (LLaMA 3 için uygun şekilde)
    if hasattr(tokenizer, "apply_chat_template"):
        full_prompt = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
    else:
        full_prompt = f"<s>[INST] {system_instruction}\n\n{user_prompt} [/INST]"

    # Tokenize edip modele ver
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)

    # Generate
    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        repetition_penalty=1.2,
    )

    # Decode et ve prompt kısmını temizle
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "[/INST]" in response:
        response = response.split("[/INST]")[-1].strip()
    else:
        response = response.replace(full_prompt, "").strip()

    return response

# Bellek temizleme
def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

# Tüm makaleleri işle
results = []
processing_times = []
start_time_total = time.time()

for i, article in enumerate(articles):
    start_time = time.time()
    print(f"Processing article {i + 1}/{len(articles)}: {article['title']}")

    try:
        abstract = generate_abstract_full_text(article)
    except Exception as e:
        print(f"Error generating abstract: {e}")
        abstract = "Error generating abstract."

    results.append({
        "title": article["title"],
        "abstract": abstract
    })

    # Süre hesaplamaları
    elapsed = time.time() - start_time
    processing_times.append(elapsed)
    avg_time = sum(processing_times) / len(processing_times)
    remaining = len(articles) - (i + 1)
    eta = datetime.now() + timedelta(seconds=avg_time * remaining)

    print(f"Done in {elapsed:.2f}s | Avg: {avg_time:.2f}s | Remaining: {remaining} | ETA: {eta.strftime('%Y-%m-%d %H:%M:%S')}")
    print("-" * 50)

    if (i + 1) % 5 == 0:
        clear_memory()

# Toplam süre
total_time = time.time() - start_time_total
print(f"All done in {total_time / 60:.2f} minutes.")

# Sonuçları kaydet
with open("Llama3.2_3B_3-5_abstracts_second_test.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

clear_memory()
print("Abstract generation completed!")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

Unsloth 2025.4.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


Processing article 1/200: discourse-aware_unsupervised_summarization_for_long_scientific_documents
Done in 18.44s | Avg: 18.44s | Remaining: 199 | ETA: 2025-05-09 16:16:16
--------------------------------------------------
Processing article 2/200: joint_geometrical_and_statistical_domain_adaptation_for_cross-domain_code_vulnerability_detection
Done in 5.73s | Avg: 12.08s | Remaining: 198 | ETA: 2025-05-09 15:55:05
--------------------------------------------------
Processing article 3/200: reward-augmented_decoding_efficient_controlled_text_generation_with_a_unidirectional_reward_model
Done in 6.79s | Avg: 10.32s | Remaining: 197 | ETA: 2025-05-09 15:49:12
--------------------------------------------------
Processing article 4/200: InterFair__Debiasing_with_Natural_Language_Feedback_for_Fair_Interpretable_Predictions
Done in 21.88s | Avg: 13.21s | Remaining: 196 | ETA: 2025-05-09 15:58:50
--------------------------------------------------
Processing article 5/200: Let’s_Think_Frame_by

...

Done in 36.85s | Avg: 20.26s | Remaining: 6 | ETA: 2025-05-09 16:22:46
--------------------------------------------------
Processing article 195/200: an_empirical_analysis_of_diversity_in_argument_summarization
Done in 12.64s | Avg: 20.22s | Remaining: 5 | ETA: 2025-05-09 16:22:38
--------------------------------------------------
Processing article 196/200: assist_automated_semantic_assistance_for_translators


Unsloth: Input IDs of length 20190 > the model's max sequence length of 16000.
We shall truncate it ourselves. It's imperative if you correct this issue first.


Done in 18.45s | Avg: 20.21s | Remaining: 4 | ETA: 2025-05-09 16:22:37
--------------------------------------------------
Processing article 197/200: interventional_rationalization


Unsloth: Input IDs of length 18860 > the model's max sequence length of 16000.
We shall truncate it ourselves. It's imperative if you correct this issue first.


Done in 14.99s | Avg: 20.19s | Remaining: 3 | ETA: 2025-05-09 16:22:32
--------------------------------------------------
Processing article 198/200: Lost_in_Translation,_Found_in_Spans__Identifying_Claims_in_Multilingual_Social_Media
Done in 9.97s | Avg: 20.14s | Remaining: 2 | ETA: 2025-05-09 16:22:21
--------------------------------------------------
Processing article 199/200: winodict_probing_language_models_for_in-context_word_acquisition
Done in 7.72s | Avg: 20.07s | Remaining: 1 | ETA: 2025-05-09 16:22:09
--------------------------------------------------
Processing article 200/200: set_learning_for_generative_information_extraction
Done in 11.97s | Avg: 20.03s | Remaining: 0 | ETA: 2025-05-09 16:22:01
--------------------------------------------------
All done in 67.22 minutes.
Abstract generation completed!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
